In [90]:
from backtesting import Backtest, Strategy
from backtesting.test import EURUSD, SMA ,XAUUSD
import numpy as np

In [91]:
EURUSD

,Open,High,Low,Close,Volume
2017-04-19 09:00:00,1.07160,1.07220,1.07083,1.07219,1413
2017-04-19 10:00:00,1.07214,1.07296,1.07214,1.07260,1241
2017-04-19 11:00:00,1.07256,1.07299,1.07170,1.07192,1025
2017-04-19 12:00:00,1.07195,1.07280,1.07195,1.07202,1460
2017-04-19 13:00:00,1.07200,1.07230,1.07045,1.07050,1554
...,...,...,...,...,...
2018-02-07 11:00:00,1.23390,1.23548,1.23386,1.23501,2203
2018-02-07 12:00:00,1.23501,1.23508,1.23342,1.23422,2325
2018-02-07 13:00:00,1.23422,1.23459,1.23338,1.23372,2824
2018-02-07 14:00:00,1.23374,1.23452,1.23238,1.23426,4065


In [92]:
def ATR(df, n):
    df = df.copy()
    df['High-Low'] = abs(df['High'] - df['Low'])
    df['High-PrevClose'] = abs(df['High'] - df['Close'].shift(1))
    df['Low-PrevClose'] = abs(df['Low'] - df['Close'].shift(1))
    df['TR'] = df[['High-Low', 'High-PrevClose',
                   'Low-PrevClose']].max(axis=1, skipna=False)
    df['ATR'] = df['TR'].rolling(n).mean()
    df = df.drop(['High-Low', 'High-PrevClose', 'Low-PrevClose'], axis=1)
    return df

In [110]:
class Momentum(Strategy) :
    def init(self):
        self.data.df['ATR'] = ATR(self.data.df, 20)['ATR']
        self.data.df['direction'] = np.where(self.data.Close > self.data.Open, 'bull', 'bear')
        self.data.df['dir_count'] = self.data.df.groupby((self.data.df['direction'] != self.data.df['direction'].shift(1)).cumsum()).cumcount()+1

    def next(self):
        current_dir_count = self.data.df.loc[self.data.df.index[-1], 'dir_count']
        current_direction = self.data.df.loc[self.data.df.index[-1], 'direction']
        current_ATR = self.data.df.loc[self.data.df.index[-1], 'ATR']
        if current_dir_count >= 5 and current_direction == 'bull' and not self.position.is_short and not self.position.is_long:
            tp = self.data.Close[-1] + current_ATR * 0.5
            sl = self.data.Open[-1] 
            limit = self.data.Open[-1]
            if tp > limit > sl:
                self.buy(sl=sl,tp=tp,limit=limit)  # Place your buy logic here
        if current_dir_count >= 5 and current_direction == 'bear' and not self.position.is_short and not self.position.is_long:
            tp = self.data.Close[-1] - current_ATR * 0.5
            sl = self.data.Open[-1] 
            limit = self.data.Open[-1]
            limit = self.data.Close[-1]
            if tp < limit < sl:
                self.sell(sl=sl, tp=tp,limit=limit)  # Place your buy logic here

In [113]:
bt = Backtest(EURUSD,Momentum)
start = bt.run()

c:\Users\Win10\anaconda3\lib\site-packages\backtesting\backtesting.py:912: UserWarning: (2017-04-21 00:00:00) A contingent SL/TP order would execute in the same bar its parent stop/limit order was turned into a trade. Since we can't assert the precise intra-candle price movement, the affected SL/TP order will instead be executed on the next (matching) price/bar, making the result (of this trade) somewhat dubious. See https://github.com/kernc/backtesting.py/issues/119
  warnings.warn(
c:\Users\Win10\anaconda3\lib\site-packages\backtesting\backtesting.py:912: UserWarning: (2017-04-21 12:00:00) A contingent SL/TP order would execute in the same bar its parent stop/limit order was turned into a trade. Since we can't assert the precise intra-candle price movement, the affected SL/TP order will instead be executed on the next (matching) price/bar, making the result (of this trade) somewhat dubious. See https://github.com/kernc/backtesting.py/issues/119
  warnings.warn(
c:\Users\Win10\anacond

In [115]:
print(start)

Start                     2017-04-19 09:00:00
End                       2018-02-07 15:00:00
Duration                    294 days 06:00:00
Exposure Time [%]                        3.58
Equity Final [$]                  9937.460165
Equity Peak [$]                       10000.0
Return [%]                          -0.625398
Buy & Hold Return [%]               14.628937
Return (Ann.) [%]                   -0.627882
Volatility (Ann.) [%]                0.956382
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                    -1.57449
Avg. Drawdown [%]                    -1.57449
Max. Drawdown Duration      292 days 16:00:00
Avg. Drawdown Duration      292 days 16:00:00
# Trades                                   75
Win Rate [%]                             40.0
Best Trade [%]                       0.280445
Worst Trade [%]                     -0.544565
Avg. Trade [%]                    

In [105]:
bt.plot()

Row(id='2427', ...)